In [1]:
f = open('toutiao_cat_data.txt', 'r', encoding='utf8')
text = []
for line in f:
    line = line.split('_!_')
    text.append(line[1:])

In [2]:
import numpy as np
import pandas as pd
news = pd.DataFrame(np.array(text)).loc[:,:2]

In [3]:
from pyhanlp import *
seg_list_all = []
for i in news[2]:
    seg_list = HanLP.segment(i)
    word_list = []
    for word_ in seg_list:
        word_list.append(word_.word)
    word_str = ' '.join(word_list)
    seg_list_all.append(word_str)
    seg_list_all.append(len(word_list))
#seg_list_all
news = pd.DataFrame(np.array(text)).loc[:,:2]
news.columns = ['label_num','label_name','content']
print(len(news))

382688


In [4]:
df_seg = pd.DataFrame(np.array(seg_list_all).reshape(382688,2))
df_seg.columns = ['seg_word','word_count']
news_all = pd.concat([news,df_seg],axis = 1)

In [5]:
train_rate = 0.4
val_rate = 0.4
test_rate =0.2
train_df = news_all.iloc[:int(train_rate*len(news))]
val_df = news_all.iloc[int(train_rate*len(news)):int((train_rate+val_rate)*len(news))]
test_df = news_all.iloc[int((train_rate+val_rate)*len(news)):]
train_df.head()

,label_num,label_name,content,seg_word,word_count
0,101,news_culture,京城最值得你来场文化之旅的博物馆,京城 最 值得 你 来 场 文化 之旅 的 博物馆,10
1,101,news_culture,发酵床的垫料种类有哪些？哪种更好？,发酵 床 的 垫料 种类 有 哪些 ？ 哪种 更好 ？,11
2,101,news_culture,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,上联 ： 黄山 黄河 黄皮肤 黄土高原 。 怎么 对 下联 ？,11
3,101,news_culture,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,林徽因 什么 理由 拒绝 了 徐志摩 而 选择 梁思成 为 终身伴侣 ？,12
4,101,news_culture,黄杨木是什么树？,黄杨 木 是 什么 树 ？,6


In [6]:
val_df.head()

,label_num,label_name,content,seg_word,word_count
153075,110,news_military,哈萨克斯坦总统纳扎尔巴耶夫在江布尔州阅兵,哈萨克斯坦 总统 纳扎尔巴耶夫 在 江布尔州 阅兵,6
153076,112,news_travel,北京西城区整治什刹海管理辖区旅游大巴乱象,北京 西城区 整治 什刹海 管理 辖区 旅游 大巴 乱象,9
153077,112,news_travel,刀剑如画：盘点十处武侠片里的美丽景色,刀剑 如 画 ： 盘点 十 处 武侠片 里 的 美丽 景色,12
153078,112,news_travel,后来的我们 牵手去阳朔,后来 的 我们 牵手 去 阳朔,7
153079,112,news_travel,这些地方动物泛滥成灾，已经威胁到了人类的生存,这些 地方 动物 泛滥成灾 ， 已经 威胁 到 了 人类 的 生存,12


In [7]:
test_df.head()

,label_num,label_name,content,seg_word,word_count
306150,109,news_tech,马云演讲 人工智能到来后，你每天只需工作4小时,马云 演讲 人工智能 到来 后 ， 你 每天 只需 工作 4 小时,13
306151,110,news_military,太霸道了：任何有美国零件的卫星都不能交给我国火箭发射,太 霸道 了 ： 任何 有 美国 零件 的 卫星 都 不能 交给 我国 火箭 发射,16
306152,110,news_military,2018年女兵征集时间已确定，只有短短42天，今年报名不一定能体检,2018 年 女兵 征集 时间 已确定 ， 只有 短短 42 天 ， 今年 报名 不一定 能 体检,17
306153,115,news_agriculture,韭菜播种难？专家有高招！,韭菜 播种 难 ？ 专家 有 高招 ！,8
306154,116,news_game,《绝地求生》：98再也不是神，只有空头才是永远的梦想,《 绝地 求生 》 ： 98 再也不是 神 ， 只有 空头 才 是 永远 的 梦想,16


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
sns.countplot(train_df.label_num)

In [9]:
print(train_df.word_count.describe())

count     153075
unique        44
top           11
freq       11210
Name: word_count, dtype: object


In [10]:
## 對dataset label 做編碼  ，  LabelEncoder可以將label分配成0~classes-1的編碼
train_y = train_df.label_name
val_y = val_df.label_name
test_y = test_df.label_name

le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1,1)
val_y = le.transform(val_y).reshape(-1,1)
test_y = le.transform(test_y).reshape(-1,1)

In [11]:
train_y

array([[ 2],
       [ 2],
       [ 2],
       ...,
       [11],
       [11],
       [ 8]])

In [12]:
## 對dataset label做one-hot encoding
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
val_y = ohe.transform(val_y).toarray()
test_y = ohe.transform(test_y).toarray()

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [13]:
train_y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
# Tokenizer可以對文本的詞進行統計次數，生成文檔詞典，以支持基於詞典位序生成文本的向量表示

## 使用Tokenizer對詞組進行編碼
## 當我們建立一個Tokenizer對象後，使用該對象的fit_on_texts()函数，以空格去辨識每個詞,
## 可以將輸入的文本中的每個詞編號，編號是根據詞頻的，詞頻越大，編號越小。

max_words = 5000
max_len = 600
tok = Tokenizer(num_words=max_words)  ## 使用的最大詞語數为5000
tok.fit_on_texts(train_df.seg_word)## 使用word_index屬性可以看到每個詞對應的編碼
## 使用word_counts屬性可以看到每個詞對應的頻數
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("===================")  
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('，', 1)
('？', 2)
('的', 3)
('！', 4)
('：', 5)
('了', 6)
('是', 7)
('你', 8)
('“', 9)
('”', 10)
('京城', 14)
('最', 5000)
('值得', 950)
('你', 14997)
('来', 2568)
('场', 728)
('文化', 472)
('之旅', 139)
('的', 60803)
('博物馆', 112)


In [15]:
# 將切割的詞轉成數字
train_seq = tok.texts_to_sequences(train_df.seg_word)
val_seq = tok.texts_to_sequences(val_df.seg_word)
test_seq = tok.texts_to_sequences(test_df.seg_word)

In [19]:
train_seq

[[36, 217, 8, 76, 286, 472, 1857, 3, 2280],
 [3009, 3, 11, 48, 2, 1830, 792, 2],
 [49, 5, 430, 4955, 225, 20, 32, 50, 2],
 [15, 2156, 600, 6, 199, 194, 82, 2],
 [7, 15, 2607, 2],
 [49, 5, 4228, 4956, 1150, 1, 20, 32, 50, 2],
 [15, 7, 196, 4557, 2],
 [1, 50, 2],
 [49, 5, 4558, 2707, 649, 1701, 1, 50, 20, 32, 2],
 [49, 5, 2555, 1922, 225, 16, 32, 50, 2],
 [2555, 1, 16, 3572],
 [7, 294, 1, 73, 734, 169, 1, 4],
 [11, 48, 55, 18, 2294, 3, 89, 2],
 [49, 1, 1296, 3976, 1, 16, 32, 50, 2],
 [1016, 1140],
 [3369, 3495, 1, 16, 3531, 266, 2],
 [49, 5, 2555, 1922, 225, 16, 32, 50, 2],
 [49, 5, 2530, 82, 4334, 1, 16, 32, 50, 2],
 [50, 185, 225, 49, 7, 2],
 [3113, 944, 43, 1, 1, 1024, 394, 3, 491, 1465],
 [812, 82, 910, 1, 376, 105, 3, 75, 31, 858, 97, 135],
 [812, 425, 82, 53, 56, 4, 812, 2295, 358, 323, 4, 53, 84, 46, 632, 4],
 [333, 498, 333, 70, 41, 2, 19, 44, 79, 6, 1, 26, 4414, 6],
 [1310, 63, 4228, 924, 778, 3629, 6, 1, 203, 329, 1, 105, 963, 14, 35, 395],
 [433, 2281, 46, 113, 1017, 1334, 3, 

In [20]:
## 將每個序列調整相同的長度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)
print(train_seq_mat.shape)
print(val_seq_mat.shape)
print(test_seq_mat.shape)

(153075, 600)
(153075, 600)
(76538, 600)


In [21]:
train_seq_mat

array([[   0,    0,    0, ..., 1857,    3, 2280],
       [   0,    0,    0, ..., 1830,  792,    2],
       [   0,    0,    0, ...,   32,   50,    2],
       ...,
       [   0,    0,    0, ...,   78, 4222,  647],
       [   0,    0,    0, ...,  726,  345, 2735],
       [   0,    0,    0, ...,    0,    0, 1009]])

In [ ]:
## LSTM模型
inputs = Input(name='inputs',shape=[max_len])
## Embedding(詞彙表大小,batch大小,每個新聞的詞長)
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(15,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(),metrics=["accuracy"])
# In[]
model_fit = model.fit(train_seq_mat,train_y,batch_size=128,epochs=10,
                      validation_data=(val_seq_mat,val_y),
                      callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] ## 當val-loss不再提升時停止訓練
                     )

W0730 15:20:48.959594 22696 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 600)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 600, 128)          640128    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
FC1 (Dense)                  (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 15)                1935      
Total params: 790,159
Trainable params: 790,159
Non-trainable params: 0
_______________________________________________________

In [ ]:
test_pre = model.predict(test_seq_mat)
## 評價預測效果，計算混淆矩陣
confm = metrics.confusion_matrix(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1))
## 混淆矩陣可視化
Labname = ["体育","娱乐","家居","房产","教育","时尚","时政","游戏","科技","财经"]
plt.figure(figsize=(8,8))
sns.heatmap(confm.T, square=True, annot=True,
            fmt='d', cbar=False,linewidths=.8,
            cmap="YlGnBu")
plt.xlabel('True label',size = 14)
plt.ylabel('Predicted label',size = 14)
plt.xticks(np.arange(10)+0.5,Labname,fontproperties = fonts,size = 12)
plt.yticks(np.arange(10)+0.3,Labname,fontproperties = fonts,size = 12)
plt.show()
print(metrics.classification_report(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1)))